In [17]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

# LLM에서 대화를 요약하거나 엔티티 추출에 자주 쓰이는 메모리이다.
# LLM을 이용하기 때문에 메모리를 실행하는데 비용이 발생한다.

# ConversationSummaryMemory는 message를 그대로 저장하는 것이 아니라 conversation의 요약을 자체적으로 해준다.
# 매우 긴 conversation일 경우 굉장히 유용하다.
# 초반에는 이전보다 더 많은 토큰과 저장공간을 차지하게된다.

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message(
    "안녕! 나는 앤더슨이야, 나는 서울에 살아",
    "와우 멋진데!",
)

In [18]:
add_message(
    "서울은 매우 아름다워",
    "나도 가고싶어!!",
)

In [19]:
get_history()

{'history': "The human greets the AI in Korean and introduces themselves as Anderson living in Seoul. The AI responds with enthusiasm, saying it's cool and expresses a desire to visit Seoul as well."}